In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [11]:
m = 10000

x = np.random.normal(size=m)
X = x.reshape((-1, 1))
y = 4. * x + 3. + np.random.normal(0, 3, size=m)

In [3]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta)) ** 2) / len(y)
    except:
        return float('inf')
def dJ(theta, X_b, y):
#     print(theta.shape)
#     print(X_b.shape)
#     print((X_b.dot(theta)).shape)
#     print(X_b.T.dot(X_b.dot(theta)).shape)
#     print(y.shape)
    return (X_b.T.dot(X_b.dot(theta) - y)) * 2 / len(y)
def gradient_descent(X_b, y, initial_theta, eta, n_iters=1e4, epilson=1e-8):
    theta = initial_theta
    i_iter = 0
    while i_iter < n_iters:
        gradient = dJ(theta, X_b, y)
#         print(gradient.shape)
        last_theta = theta
        theta = theta - eta * gradient
        if abs(J(theta, X_b, y) -J(last_theta, X_b, y)) < epilson:   # !!!!!sgd不需要
            break
        i_iter += 1
    return theta

In [4]:
%%time 
X_b = np.hstack([np.ones((len(X), 1)), X])

initial_theta = np.zeros(X_b.shape[1])
eta = 0.1
theta = gradient_descent(X_b, y, initial_theta, eta)

CPU times: user 479 ms, sys: 695 ms, total: 1.17 s
Wall time: 41.3 ms


# SGD

In [5]:
def dJ_sgd(theta, X_b_i, y_i):
    return X_b_i.T.dot(X_b_i.dot(theta) - y_i) * 2

In [6]:
def sgd(X_b, y, initial_theta, n_iters):
    t0 = 5
    t1 = 50
    
    def learning_rates(t):
        return t0 / (t + t1)
    
    theta = initial_theta
    for i_iter in range(n_iters):
        rand_i = np.random.randint(len(X_b))
        gradient = dJ_sgd(theta, X_b[rand_i], y[rand_i])
        theta = theta - learning_rates(i_iter) * gradient
    return theta
    

In [7]:
%%time
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1])
theta = sgd(X_b, y, initial_theta, n_iters = len(y) // 3)
theta

CPU times: user 49.7 ms, sys: 1.44 ms, total: 51.1 ms
Wall time: 50.8 ms


array([3.08380954, 3.95474533])

# use my_ml

In [12]:
from my_ml.LinearRegression import LinearRegression

In [13]:
lin_reg = LinearRegression()
lin_reg.fit_sgd(X, y, n_iters=2)

Linearegression()

In [14]:
lin_reg._theta

array([3.03049051, 4.00308428])

# use public datasets

In [15]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

boston = datasets.load_boston()

X = boston.data
y = boston.target

X = X[y < 50.0]
y = y[y < 50.0]

from my_ml.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_ratio=0.2, seed=666)

In [16]:
# standarlization
from sklearn.preprocessing import StandardScaler
standarscaler = StandardScaler()
standarscaler.fit(X_train, y_train)

X_train_standard = standarscaler.transform(X_train)
X_test_standard = standarscaler.transform(X_test)

In [18]:
from my_ml.LinearRegression import LinearRegression
reg = LinearRegression()
%timeit reg.fit_sgd(X_train_standard, y_train, n_iters=2)
reg.score(X_test_standard, y_test)

5.17 ms ± 26.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


0.6527921572751846

In [19]:
%timeit reg.fit_sgd(X_train_standard, y_train, n_iters=50)
reg.score(X_test_standard, y_test)

127 ms ± 1.94 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


0.8099338873295959

In [20]:
%timeit reg.fit_sgd(X_train_standard, y_train, n_iters=100)
reg.score(X_test_standard, y_test)

252 ms ± 9.14 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


0.8107852205867202

# scikit-learn

In [21]:
from sklearn.linear_model import SGDRegressor

In [29]:
sgd_reg = SGDRegressor()
%timeit sgd_reg.fit(X_train_standard, y_train)
sgd_reg.score(X_test_standard, y_test)

1.89 ms ± 71.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


0.8120882953346749

In [28]:
sgd_reg2 = SGDRegressor(max_iter=100)
%timeit sgd_reg2.fit(X_train_standard, y_train)
sgd_reg2.score(X_test_standard, y_test)

1.89 ms ± 84.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


0.8127778418987318

In [26]:
help(SGDRegressor())

Help on SGDRegressor in module sklearn.linear_model._stochastic_gradient object:

class SGDRegressor(BaseSGDRegressor)
 |  Linear model fitted by minimizing a regularized empirical loss with SGD
 |  
 |  SGD stands for Stochastic Gradient Descent: the gradient of the loss is
 |  estimated each sample at a time and the model is updated along the way with
 |  a decreasing strength schedule (aka learning rate).
 |  
 |  The regularizer is a penalty added to the loss function that shrinks model
 |  parameters towards the zero vector using either the squared euclidean norm
 |  L2 or the absolute norm L1 or a combination of both (Elastic Net). If the
 |  parameter update crosses the 0.0 value because of the regularizer, the
 |  update is truncated to 0.0 to allow for learning sparse models and achieve
 |  online feature selection.
 |  
 |  This implementation works with data represented as dense numpy arrays of
 |  floating point values for the features.
 |  
 |  Read more in the :ref:`User 